In [1]:
from dp_model.model_files.sfcn import SFCN
from dp_model import dp_loss as dpl
from dp_model import dp_utils as dpu
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import nibabel as nib
import pandas as pd
from tqdm.autonotebook import tqdm
from sklearn.metrics import mean_absolute_error
import shutil
import zipfile

In [2]:
print(torch.__version__)

1.11.0+cu102


In [3]:
model = SFCN()
model = torch.nn.DataParallel(model)
fp_ = './brain_age/run_20190719_00_epoch_best_mae.p'
model.load_state_dict(torch.load(fp_))
model.cuda()

DataParallel(
  (module): SFCN(
    (feature_extractor): Sequential(
      (conv_0): Sequential(
        (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): ReLU()
      )
      (conv_1): Sequential(
        (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): ReLU()
      )
      (conv_2): Sequential(
        (0): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, 

In [4]:
path = '/home/jruffle/Desktop/brc2/'
metadata_test = pd.read_csv(path+'james/BIOBANK/TEST/metadata_test.csv',index_col=0)
metadata_test=metadata_test[::2].reset_index(drop=True) 
metadata_test['SCFN_age_prediction']=0
metadata_test['SCFN_age_prediction_KL']=0

In [5]:
for i, row in tqdm(metadata_test.iterrows(),total=metadata_test.shape[0]):
    zip_t1 = sorted(glob.glob(path+'Biobank/biobank_T1/'+str(int(row['biobank_id']))+'*.zip'))[0]
    
    with zipfile.ZipFile(zip_t1) as z:
        with z.open('T1/T1_brain_to_MNI.nii.gz') as zf, open('/home/jruffle/Desktop/T1_temp.nii.gz', 'wb') as f:
            shutil.copyfileobj(zf, f)
    
    data = np.asanyarray(nib.load('/home/jruffle/Desktop/T1_temp.nii.gz').dataobj)
    
    label = row['age']

    # Transforming the age to soft label (probability distribution)
    bin_range = [42,82]
    bin_step = 1
    sigma = 1
    y, bc = dpu.num2vect(label, bin_range, bin_step, sigma)
    y = torch.tensor(y, dtype=torch.float32)

    # Preprocessing
    data = data/data.mean()
    data = dpu.crop_center(data, (160, 192, 160))

    # Move the data from numpy to torch tensor on GPU
    sp = (1,1)+data.shape
    data = data.reshape(sp)
    input_data = torch.tensor(data, dtype=torch.float32).cuda()
    
    # Evaluation
    model.eval() # Don't forget this. BatchNorm will be affected if not in eval mode.
    with torch.no_grad():
        output = model(input_data)

    # Output, loss, visualisation
    x = output[0].cpu().reshape([1, -1])
    loss = dpl.my_KLDivLoss(x, y).numpy()

    # Prediction, Visualisation and Summary
    x = x.numpy().reshape(-1)
    y = y.numpy().reshape(-1)

    prob = np.exp(x)
    pred = prob@bc

    metadata_test.loc[i,'SCFN_age_prediction']=pred
    metadata_test.loc[i,'SCFN_age_prediction_KL']=loss
    os.remove('/home/jruffle/Desktop/T1_temp.nii.gz')

  0%|          | 0/2381 [00:00<?, ?it/s]

In [7]:
print(mean_absolute_error(metadata_test['age'], metadata_test['SCFN_age_prediction']))

5.045779683905161
